# CSC 480-F25 Lab 6: Knowledge Graphs

### Author:
***Arnav Bhola, Pranav Krishna***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview

This lab covers Knowledge Graphs with Neo4j—from standing up a local graph database and writing basic Cypher, to designing a practical schema with constraints and indexes, then (optionally) using an agentic system to help plan and create that schema. You’ll work with a small investigative dataset (people, events, locations, evidence, and cases) and translate flat CSVs into a connected graph that supports rich querying and reasoning.

Specifically, you will:
- Install and connect to Neo4j Desktop; verify connectivity using the Python driver.
- Inspect the provided CSVs and understand how entities (nodes) and relations (edges) map to a graph model.
- Create an example schema via Cypher with unique constraints and helpful indexes, using a provided helper for executing queries.
- Optionally, invoke an AutoGen-based agentic workflow (Azure OpenAI) to propose and materialize the schema automatically.
- Prepare the knowledge graph you’ll query and reason over in the next lab.

By the end, you should be able to model a domain as a graph, set up constraints and indexes to keep it clean and fast, load/validate data with Cypher, and set the stage for reasoning and querying in Lab 7.

---

## Part 1: Setting up Neo4j (Desktop)

### Installation

Follow the instructions at this [link](https://neo4j.com/docs/desktop/current/installation/). Install version 2.0.5 for whatever OS your system uses. You'll have to create an account.

### Creating a Graph DB with Neo4j desktop

Open the Neo4j desktop application and follow the directions to create your first Neo4j instance.
This instance acts as your DBMS, which manages Graph DBs.
Then, follow the instructrions [here](https://neo4j.com/docs/desktop/current/operations/database-management/) to create and host a Graph DB on your system with local host. When you create a DB you'll be prompted to create a password for the DB. It can be as simple as you'd like, as it's only accessible by users on your system, but you need to remember it.


In [1]:
%pip install pandas neo4j neomodel "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------  11.3/11.3 MB 101.0 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.3 MB 101.0 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 26.3 MB/s  0:00:00
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------------------------  12.8/13.1 MB 114.4 MB/s eta 0:00:01
   ---------------------------------------  12.8/13.1 MB 114.4 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 21.0 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/6 [pytz]
   ---------------------------------------- 0/6 [pytz]
   -----------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Sanity check that the database is set up correctly
from neo4j import GraphDatabase

username = "neo4j"  # the default user created when you set up Neo4j
password = "RandomPassword"  # example password
hostname = "127.0.0.1"
port = 7687  # default Bolt protocol port
uri = f"bolt://{hostname}:{port}"

# Test with the native neo4j driver to see if we get more details
try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run("RETURN 1 as test")
        print(f"Connection successful! Result: {result.single()['test']}")
    driver.close()
except Exception as e:
    print(f"Connection failed: {type(e).__name__}: {e}")

Connection successful! Result: 1


In [4]:
# Extract the data files
!unzip -o L6-7_data.zip -d ./L6-7_data

Archive:  L6-7_data.zip
 extracting: ./L6-7_data/Case.csv    
  inflating: ./L6-7_data/Case_Related_Rel.csv  
  inflating: ./L6-7_data/Event.csv   
  inflating: ./L6-7_data/Event_Evidence_Location_Rel.csv  
  inflating: ./L6-7_data/Evidence.csv  
  inflating: ./L6-7_data/Location.csv  
  inflating: ./L6-7_data/Person.csv  
  inflating: ./L6-7_data/Person_Location_Rel.csv  
  inflating: ./L6-7_data/Person_Person_Rel.csv  


### L6-7_data: High-level dataset breakdown

This dataset models the Kristin Smart case with clearly separated node CSVs (entities) and relationship CSVs (edges), using Neo4j-style import headers. Here’s the global picture and what each file contributes.

#### Entities (nodes)

- Person.csv
  - Key columns: `id`, `name`, `type`, `status`, `dob`
  - Examples: KS (Kristin Smart – Victim), PF (Paul Flores – Suspect/Murderer), RF (Ruben Flores – Accessory), family and witnesses.
  - Notes: `type` captures role (Victim, Suspect, Family, Witness); `status` captures lifecycle (e.g., Convicted, Deceased, Key Witness).

- Location.csv
  - Key columns: `id`, `name`, `type`, `address`, `city`
  - Examples: PARTY_LOC (Crandall Way party house), MUIR_HALL, SANTA_LUCIA (dorms), RF_HOME, PF_HOME_LA, EXCAVATION_16 (dig site), MONTEREY_COURT.
  - Notes: `type` distinguishes residences, dorms, search sites, and venues.

- Event.csv
  - Key columns: `id`, `type`, `date`, `description`
  - Examples: DISAPPEAR (last sighting), DECLARE_DEAD, DIG_2016, SEARCH_RF_HOME, ARRESTS, TRIAL_START, PF_GUILTY.
  - Notes: Seminal milestones with dates and human-readable descriptions.

- Evidence.csv
  - Key columns: `id`, `type`, `status`, `description`
  - Examples: EARING (lost by police), TRUCKS (seized/analyzed), DRUGS (found at PF_HOME_LA), BIOLOGICAL (under RF deck), VOLKSWAGEN, VIDEOS.
  - Notes: `status` reflects chain-of-custody or analysis (Seized/Analyzed, Unknown, etc.).

- Case.csv
  - Key columns: `id`, `name`, `status`, `dateOpened`
  - Example: CASE_KS (Murder of Kristin Smart; status shows conviction and the case open date).

#### Relationships (edges)

- Person_Person_Rel.csv (Person → Person)
  - Columns: `:START_ID(Person)`, `:END_ID(Person)`, `:TYPE`, `relationshipType`
  - Examples: `ACCOMPANIED_BY` (who walked with whom), `FAMILY_RELATIONSHIP` (e.g., PF → RF Father).
  - Notes: `relationshipType` adds semantic detail (e.g., "Last Known Person", "Spouse").

- Person_Location_Rel.csv (Person → Location)
  - Columns: `:START_ID(Person)`, `:END_ID(Location)`, `:TYPE`, `date`, `time`
  - Examples: `ATTENDED_PARTY_AT` (KS/PF/others → PARTY_LOC), `LAST_SEEN_NEAR` (KS → SANTA_LUCIA), `LIVED_AT`/`RESIDENCE_OF` for dorm/home ties.
  - Notes: `date`/`time` may be missing for some records; treat as optional properties.

- Event_Evidence_Location_Rel.csv (Evidence → Location)
  - Columns: `Event_Evidence_Location_Rel:START_ID`, `:END_ID`, `:TYPE`, `date`
  - Examples: `FOUND_AT` (BIOLOGICAL → RF_HOME), `SEIZED_FROM` (VOLKSWAGEN/DRUGS/VIDEOS → RF_HOME/PF_HOME_LA) with dates.
  - Notes: Despite the header name, the `START_ID` values correspond to Evidence IDs (e.g., DRUGS, VOLKSWAGEN). Use as Evidence → Location.

- Case_Related_Rel.csv (Person → Case)
  - Columns: `:START_ID(Person)`, `:END_ID(Case)`, `:TYPE`, `outcome`
  - Examples: `VICTIM_IN` (KS → CASE_KS), `SUSPECT_IN` (PF → CASE_KS, outcome=Convicted), `ACCUSED_IN` (RF → CASE_KS, outcome=Acquitted), `FILED_CIVIL_SUIT_IN` (family → CASE_KS).
  - Notes: `outcome` is optional and captures legal results when present.

In [6]:
from pathlib import Path
import pandas as pd

data_path = Path("./L6-7_data")
data = [(file.name, pd.read_csv(file)) for file in data_path.glob("*.csv")]

data_str = ""
for name, df in data:
    data_str += f"File: {name}\nDataframe head:{df}\n" + "=" * 90 + "\n\n"

# The names and dataframes for each file
print(data_str)

File: Case.csv
Dataframe head:        id                     name               status  dateOpened
0  CASE_KS  Murder of Kristin Smart  Solved (Conviction)  1996-05-27

File: Case_Related_Rel.csv
Dataframe head:  :START_ID(Person) :END_ID(Case)                :TYPE    outcome
0                KS       CASE_KS            VICTIM_IN        NaN
1                PF       CASE_KS           SUSPECT_IN  Convicted
2                RF       CASE_KS           ACCUSED_IN  Acquitted
3                DS       CASE_KS  FILED_CIVIL_SUIT_IN        NaN
4                SS       CASE_KS  FILED_CIVIL_SUIT_IN        NaN

File: Event.csv
Dataframe head:               id                           type                 date  \
0       DISAPPEAR    Last Sighting/Disappearance  1996-05-25 02:00:00   
1    DECLARE_DEAD          Legally Declared Dead           2002-05-25   
2        DIG_2016            Cal Poly Excavation           2016-09-06   
3  SEARCH_RF_HOME  Search Warrant (Ruben Flores)           2021-03-15

---

## Part 2: An Example Knowledge Graph Schema

The following creates an example knowledge graph schema, and instantiates it in your Neo4j graph DB instance.

### First, a tool for executing Cypher queries in Neo4j

In [9]:
from neo4j import GraphDatabase


# Tool function to execute Cypher queries
def execute_cypher_query(query_str: str, description: str = "Executing query", verbose: bool = False) -> str:
    """
    Executes a Cypher query on the Neo4j database.

    Args:
        query_str: The Cypher query to execute
        description: A description of what the query does

    Returns:
        A string describing the result of the query execution
    """
    if verbose:
        print(f"\n{'='*80}")
        print(f"EXECUTING CYPHER QUERY: {description}")
        print(f"{'='*80}")
        print(f"Query:\n{query_str}")
        print(f"{'='*80}\n")

    # Parse the query into individual statements if needed
    queries = " ".join(
        [q for q in query_str.splitlines() if not q.strip().startswith("//")]
    )
    queries = [q.strip() + ";" for q in queries.split(";") if q.strip()]

    nodes_created = 0
    relationships_created = 0
    properties_set = 0
    labels_added = 0
    indexes_added = 0
    constraints_added = 0
    response_parts = []

    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        for query in queries:
            try:
                result = session.run(query)
                summary = result.consume()

                nodes_created += summary.counters.nodes_created
                relationships_created += summary.counters.relationships_created
                properties_set += summary.counters.properties_set
                labels_added += summary.counters.labels_added
                indexes_added += summary.counters.indexes_added
                constraints_added += summary.counters.constraints_added

            except Exception as e:
                error_msg = f"Error executing query: {type(e).__name__}: {str(e)}"
                print(error_msg)
                if "driver" in locals():
                    driver.close()
                return error_msg
    driver.close()
    response_parts.append(f"Nodes created: {nodes_created}")
    response_parts.append(f"Relationships created: {relationships_created}")
    response_parts.append(f"Properties set: {properties_set}")
    response_parts.append(f"Labels added: {labels_added}")
    response_parts.append(f"Indexes added: {indexes_added}")
    response_parts.append(f"Constraints added: {constraints_added}")

    response = "\n".join(response_parts)
    
    if verbose:
        print(f"\n{'='*80}")
        print(f"QUERY EXECUTION COMPLETE")
        print(f"{'='*80}\n")
        print(response)

    return response

### An example schema

This just sets up the knowledge graph structure, it doesn't ingest any data.

In [12]:
query_str = """
// === NODE CONSTRAINTS (IDs unique + present) ===
CREATE CONSTRAINT case_id_unique IF NOT EXISTS
FOR (c:Case) REQUIRE c.id IS UNIQUE;
CREATE CONSTRAINT case_id_exists IF NOT EXISTS
FOR (c:Case) REQUIRE c.id IS NOT NULL;
CREATE CONSTRAINT event_id_unique IF NOT EXISTS
FOR (e:Event) REQUIRE e.id IS UNIQUE;
CREATE CONSTRAINT event_id_exists IF NOT EXISTS
FOR (e:Event) REQUIRE e.id IS NOT NULL;
CREATE CONSTRAINT evidence_id_unique IF NOT EXISTS
FOR (ev:Evidence) REQUIRE ev.id IS UNIQUE;
CREATE CONSTRAINT evidence_id_exists IF NOT EXISTS
FOR (ev:Evidence) REQUIRE ev.id IS NOT NULL;
CREATE CONSTRAINT location_id_unique IF NOT EXISTS
FOR (l:Location) REQUIRE l.id IS UNIQUE;
CREATE CONSTRAINT location_id_exists IF NOT EXISTS
FOR (l:Location) REQUIRE l.id IS NOT NULL;
CREATE CONSTRAINT person_id_unique IF NOT EXISTS
FOR (p:Person) REQUIRE p.id IS UNIQUE;
CREATE CONSTRAINT person_id_exists IF NOT EXISTS
FOR (p:Person) REQUIRE p.id IS NOT NULL;

// === RELATIONSHIP PROPERTY EXISTENCE (Enterprise Edition) ===
// Person-Person
CREATE CONSTRAINT accompanied_by_relationshipType_exists IF NOT EXISTS
FOR ()-[r:ACCOMPANIED_BY]-() REQUIRE r.relationshipType IS NOT NULL;

// Person-Location (carry dates; time may be optional)
CREATE CONSTRAINT attended_party_at_date_exists IF NOT EXISTS
FOR ()-[r:ATTENDED_PARTY_AT]-() REQUIRE r.date IS NOT NULL;
CREATE CONSTRAINT last_seen_near_date_exists IF NOT EXISTS
FOR ()-[r:LAST_SEEN_NEAR]-() REQUIRE r.date IS NOT NULL;
CREATE CONSTRAINT lived_at_date_exists IF NOT EXISTS
FOR ()-[r:LIVED_AT]-() REQUIRE r.date IS NOT NULL;
CREATE CONSTRAINT residence_of_date_exists IF NOT EXISTS
FOR ()-[r:RESIDENCE_OF]-() REQUIRE r.date IS NOT NULL;

// Evidence-Location (carry dates)
CREATE CONSTRAINT found_at_date_exists IF NOT EXISTS
FOR ()-[r:FOUND_AT]-() REQUIRE r.date IS NOT NULL;
CREATE CONSTRAINT seized_from_date_exists IF NOT EXISTS
FOR ()-[r:SEIZED_FROM]-() REQUIRE r.date IS NOT NULL;

// Case-related edges typically include optional outcome metadata
CREATE INDEX person_name IF NOT EXISTS FOR (p:Person) ON (p.name);
CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name);
CREATE INDEX case_name IF NOT EXISTS FOR (c:Case) ON (c.name);
"""

execute_cypher_query(query_str, description="Creating knowledge graph schema", verbose=True)


EXECUTING CYPHER QUERY: Creating knowledge graph schema
Query:

// === NODE CONSTRAINTS (IDs unique + present) ===
CREATE CONSTRAINT case_id_unique IF NOT EXISTS
FOR (c:Case) REQUIRE c.id IS UNIQUE;
CREATE CONSTRAINT case_id_exists IF NOT EXISTS
FOR (c:Case) REQUIRE c.id IS NOT NULL;
CREATE CONSTRAINT event_id_unique IF NOT EXISTS
FOR (e:Event) REQUIRE e.id IS UNIQUE;
CREATE CONSTRAINT event_id_exists IF NOT EXISTS
FOR (e:Event) REQUIRE e.id IS NOT NULL;
CREATE CONSTRAINT evidence_id_unique IF NOT EXISTS
FOR (ev:Evidence) REQUIRE ev.id IS UNIQUE;
CREATE CONSTRAINT evidence_id_exists IF NOT EXISTS
FOR (ev:Evidence) REQUIRE ev.id IS NOT NULL;
CREATE CONSTRAINT location_id_unique IF NOT EXISTS
FOR (l:Location) REQUIRE l.id IS UNIQUE;
CREATE CONSTRAINT location_id_exists IF NOT EXISTS
FOR (l:Location) REQUIRE l.id IS NOT NULL;
CREATE CONSTRAINT person_id_unique IF NOT EXISTS
FOR (p:Person) REQUIRE p.id IS UNIQUE;
CREATE CONSTRAINT person_id_exists IF NOT EXISTS
FOR (p:Person) REQUIRE p.id

'Nodes created: 0\nRelationships created: 0\nProperties set: 0\nLabels added: 0\nIndexes added: 0\nConstraints added: 0'

---

## Part 3: Or ask an agentic system to build it for us!

Note: this is not guaranteed to converge, and it may take a long time.

In [15]:
import os
from typing import Any
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from neo4j import GraphDatabase

# Configure Azure OpenAI client
azure_deployment = "gpt-5-mini"  # Replace with your deployment name
api_version = "2024-12-01-preview"
azure_endpoint = os.getenv("AZURE_ENDPOINT")  # Replace with your endpoint
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")  # Set this environment variable

if not api_key:
    raise ValueError("AZURE_OPENAI_API_KEY environment variable not set.")

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini",
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

schema_planner_message = """
You are a Graph Database Schema Planner. Your job is to analyze the provided data 
and design an optimal graph database schema for Neo4j.

Given the data structure, you should:
1. Identify which CSV files should become node types
2. Identify which CSV files represent relationships between nodes
3. Plan constraints and indexes for efficient querying
4. Explain your reasoning clearly
5. Consider feedback from the User Proxy

Files ending in "_Rel.csv" typically represent relationships between entities.
Files without "_Rel" typically represent node entities.

Once you have planned the schema and the User Proxy approves, say "SCHEMA_READY" to proceed.
"""

schema_creator_message = """
You are a Graph Database Schema Creator. You receive schema plans and create them
in Neo4j using Cypher queries.

Your tasks:
1. Clear any existing data (use MATCH (n) DETACH DELETE n)
2. Create constraints for unique identifiers (use CREATE CONSTRAINT ... IF NOT EXISTS)
3. Create indexes for frequently queried properties

Use the execute_query tool to run your cypher queries. Provide clear descriptions
of what each query does.

After creating the schema successfully, say "SCHEMA_CREATED" to finish.
"""

schema_planner = AssistantAgent(
    name="SchemaPlanner",
    system_message=schema_planner_message,
    model_client=client,
)

# -- FIX: Use a properly-typed function for the tool, not a lambda, and provide type hints.

def query_tool(
    query_str: str, 
    description: str,
    verbose: bool = True
) -> Any:
    """
    Executes a Cypher query with a description and verbosity flag.
    """
    return execute_cypher_query(
        query_str=query_str,
        description=description,
        verbose=verbose
    )

# Register the tool function with correct signature
schema_creator = AssistantAgent(
    name="SchemaCreator",
    system_message=schema_creator_message,
    model_client=client,
    tools=[query_tool],
)


# Run the agentic system
async def build_knowledge_graph():
    """
    Runs the agentic system to plan and create schema in Neo4j using round robin.
    """
    print("\n" + "=" * 80)
    print("STARTING AGENTIC KNOWLEDGE GRAPH SCHEMA DESIGN")
    print("=" * 80 + "\n")

    termination = TextMentionTermination("SCHEMA_CREATED")
    groupchat = RoundRobinGroupChat(
        [schema_planner, schema_creator],
        max_turns=100,
        termination_condition=termination,
    )

    # Create the initial task with the data_str (from earlier cell) context
    task = f"""
    We need to design and create a Knowledge Graph schema in Neo4j from the following data:

    {data_str}

    UserProxy: Please review the data and guide the schema design process.
    SchemaPlanner: Analyze this data and design an optimal graph schema.
    SchemaCreator: Once the schema is approved, create it in Neo4j.

    Work together in round robin fashion to complete this task step by step.
    """

    result: TaskResult = await groupchat.run(task=task)

    print("\n" + "=" * 80)
    print("KNOWLEDGE GRAPH SCHEMA DESIGN COMPLETE")
    print("=" * 80 + "\n")

    return result


# Execute the agentic system
result = await build_knowledge_graph()

# Print out all messages from the agents, if any
if result is not None:
    for message in result.messages:
        print(f"{message.content}\n{'='*80}\n")


STARTING AGENTIC KNOWLEDGE GRAPH SCHEMA DESIGN


EXECUTING CYPHER QUERY: 1) Clear the database of all nodes and relationships.
2) Create uniqueness constraints for id on Person, Case, Event, Evidence, and Location labels (if they don't already exist).
3) Create indexes to speed common lookups: Person.name, Person.type, Person.dob, Location.name, Case.name, Event.date, Evidence.status, Evidence.type.
Query:
MATCH (n) DETACH DELETE n;

CREATE CONSTRAINT person_id_unique IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE;
CREATE CONSTRAINT case_id_unique IF NOT EXISTS FOR (c:Case) REQUIRE c.id IS UNIQUE;
CREATE CONSTRAINT event_id_unique IF NOT EXISTS FOR (e:Event) REQUIRE e.id IS UNIQUE;
CREATE CONSTRAINT evidence_id_unique IF NOT EXISTS FOR (ev:Evidence) REQUIRE ev.id IS UNIQUE;
CREATE CONSTRAINT location_id_unique IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE;

CREATE INDEX person_name_idx IF NOT EXISTS FOR (p:Person) ON (p.name);
CREATE INDEX person_type_idx IF NOT EXISTS FO

---

## Part 4: Reflection

##### What worked well in the schema creation and data inspection?

The execute_cypher_query helper function worked exceptionally well by providing clear feedback on nodes created, relationships created, properties set, and constraints/indexes added, which made debugging and verification straightforward. The Neo4j Python driver connection was smooth and reliable, and using pandas to preview the CSVs made it easy to understand the data structure and identify node types versus relationships. The IF NOT EXISTS clause in Cypher allowed safe re-running of schema creation queries during iterative development, and the logical separation of node entities from relationship CSVs made the graph model intuitive and aligned well with Neo4j's property graph paradigm.

##### What struggled?

The Agentic Schema Creation took a long time to run about 9 minutes and we were unsure if the algorithm would properly converge, but it did provide a result and it successfully created the schemas. We had trouble getting this to work since we were getting a key error with the current code, so some code changes were required for the algorithm to run.

##### Manual Cypher vs. Agentic Schema Creation

Manual Cypher provides full control and transparency with faster execution for small schemas, high reproducibility, and easier debugging, making it ideal for production systems where correctness and performance are critical. Agentic schema creation offers less transparency and slower performance due to LLM inference latency with potential non-deterministic results, but can be useful for exploring new domains or rapid prototyping. The practical recommendation is to use manual Cypher for production systems and consider agentic approaches only for educational exploration or prototyping, with a hybrid approach of using LLM-generated suggestions that you manually review and refine potentially offering the best balance.